In [1]:
%matplotlib inline
import pickle as pk
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Load data
file_path = Path("Resources/pickled_raws.pkl")
df = pd.read_pickle(file_path)
df.head()

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,msamd,state_code,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,action_taken_summary,Region
respondent_id,,,,,,,,,,,,,,,,,,,,,
0000342634,CFPB,Conventional,One to Four-Family,Refinancing,Owner Occupied,9.0,Not Applicable,Application Denied by Financial Institution,33860.0,AL,...,Not a HOEPA Loan,Secured by a Subordinate Loan,1948.0,12.58,59700.0,122.93,507.0,724.0,0,SE
0004186591,CFPB,VA-Guaranteed,One to Four-Family,Home Purchase,Owner Occupied,94.0,Not Applicable,Loan Purchased by the Institution,33860.0,AL,...,Not a HOEPA Loan,Not Applicable (Purchased Loan),1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE
22-3039688,HUD,VA-Guaranteed,One to Four-Family,Refinancing,Owner Occupied,161.0,Not Applicable,File Closed for Incompleteness,33860.0,AL,...,Not a HOEPA Loan,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,2,SE
20-3878295,NCUA,Conventional,One to Four-Family,Home Improvement,Owner Occupied,90.0,Not Applicable,Loan Originated,33860.0,AL,...,Not a HOEPA Loan,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE
0000342634,CFPB,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,41.0,Not Applicable,Loan Originated,33860.0,AL,...,Not a HOEPA Loan,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE


In [3]:
# Drop unneccessary columns
df = df.drop(columns=["action_taken", "purchaser_type", "denial_reason_1", "state_code", "hoepa_status"])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13997124 entries, 0000342634 to 0000510871
Data columns (total 26 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   agency_code                     object 
 1   loan_type                       object 
 2   property_type                   object 
 3   loan_purpose                    object 
 4   owner_occupancy                 object 
 5   loan_amount_000s                float64
 6   preapproval                     object 
 7   msamd                           float64
 8   county_code                     float64
 9   census_tract_number             float64
 10  applicant_ethnicity             object 
 11  co_applicant_ethnicity          object 
 12  applicant_race_1                object 
 13  co_applicant_race_1             object 
 14  applicant_sex                   object 
 15  co_applicant_sex                object 
 16  applicant_income_000s           float64
 17  lien_status        

In [5]:
# Remove rows containing N/A, incomplete, or irrelevant information
df = df.loc[df["loan_purpose"] == "Home Purchase"]
df = df.drop(columns="loan_purpose")

In [6]:
df = df.loc[df["owner_occupancy"] != "Not Applicable"]

In [7]:
df = df.loc[df["applicant_sex"] != "Information not Provided"]

In [8]:
df = df.loc[df["applicant_sex"] != "Not Applicable"]

In [9]:
df = df.loc[df["co_applicant_sex"] != "Information not Provided"]
df = df.loc[df["co_applicant_sex"] != "Not Applicable"]

In [10]:
df = df.loc[df["applicant_ethnicity"] != "Not Applicable"]

In [11]:
df = df.loc[df["co_applicant_ethnicity"] != "Not Applicable"]

In [12]:
# Remove outlier homes
df = df.loc[df["loan_amount_000s"] <= 1650]

In [13]:
# Restrict target variable to only approvals and denials
df = df.loc[df["action_taken_summary"] != 2]

In [14]:
df.count()

agency_code                       5149278
loan_type                         5149278
property_type                     5149278
owner_occupancy                   5149278
loan_amount_000s                  5149278
preapproval                       5149278
msamd                             5149278
county_code                       5149278
census_tract_number               5149278
applicant_ethnicity               5149278
co_applicant_ethnicity            5149278
applicant_race_1                  5149278
co_applicant_race_1               5149278
applicant_sex                     5149278
co_applicant_sex                  5149278
applicant_income_000s             5149278
lien_status                       5149278
population                        5149278
minority_population               5149278
hud_median_family_income          5149278
tract_to_msamd_income             5149278
number_of_owner_occupied_units    5149278
number_of_1_to_4_family_units     5149278
action_taken_summary              

In [15]:
df.head()

,agency_code,loan_type,property_type,owner_occupancy,loan_amount_000s,preapproval,msamd,county_code,census_tract_number,applicant_ethnicity,...,applicant_income_000s,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,action_taken_summary,Region
respondent_id,,,,,,,,,,,,,,,,,,,,,
0000013303,FDIC,FHA-Insured,One to Four-Family,Owner Occupied,76.0,Not Applicable,33860.0,1.0,201.0,Not Hispanic or Latino,...,87.0,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE
0000233031,CFPB,FHA-Insured,One to Four-Family,Owner Occupied,92.0,Preapproval Requested,33860.0,1.0,201.0,Not Hispanic or Latino,...,29.0,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE
27-0349391,HUD,VA-Guaranteed,One to Four-Family,Owner Occupied,94.0,Preapproval Not Requested,33860.0,1.0,201.0,Not Hispanic or Latino,...,53.0,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE
62-0997810,HUD,Conventional,Manufactured,Owner Occupied,77.0,Not Applicable,33860.0,1.0,201.0,Info not Provided by Applicant,...,54.0,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,0,SE
63-1005978,HUD,FHA-Insured,One to Four-Family,Owner Occupied,147.0,Preapproval Not Requested,33860.0,1.0,201.0,Not Hispanic or Latino,...,90.0,Secured by First Lien,1948.0,12.58,59700.0,122.93,507.0,724.0,1,SE


In [16]:
# Drop index
df = df.reset_index(drop=True)

In [17]:
# Generate our categorical variable lists
action_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[action_cat].nunique()

agency_code               6
loan_type                 4
property_type             3
owner_occupancy           2
preapproval               3
applicant_ethnicity       3
co_applicant_ethnicity    4
applicant_race_1          6
co_applicant_race_1       7
applicant_sex             2
co_applicant_sex          3
lien_status               3
Region                    7
dtype: int64

In [18]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[action_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(action_cat)
encode_df.head()

C:\Users\sherr\anaconda3\envs\ml2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,agency_code_CFPB,agency_code_FDIC,agency_code_FRS,agency_code_HUD,agency_code_NCUA,agency_code_OCC,loan_type_Conventional,loan_type_FHA-Insured,loan_type_FSA/RHS,loan_type_VA-Guaranteed,...,lien_status_Not Applicable (Purchased Loan),lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
# Merge dummy columns and drop originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(action_cat, axis=1)
df.count()

loan_amount_000s         5149278
msamd                    5149278
county_code              5149278
census_tract_number      5149278
applicant_income_000s    5149278
                          ...   
Region_NE                5149278
Region_NW                5149278
Region_SE                5149278
Region_SW                5149278
Region_W                 5149278
Length: 65, dtype: int64

In [20]:
# Define features and targets
y = df["action_taken_summary"]
X = df.drop(columns="action_taken_summary")

In [21]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21)

In [22]:
# Create StandardScaler instance, fit to X, scale
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Create a random forest classifier
model = RandomForestClassifier(max_depth = 2, max_features = 3, n_estimators=100, random_state=21)

In [ ]:
# Fit model
model = model.fit(X_train_scaled, y_train)

In [ ]:
predictions = model.predict(X_test_scaled)

In [ ]:
# Calculate confusion matrix and accuracy score
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index= ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score: {acc_score}")
print(classification_report(y_test, predictions))

In [ ]:
# Sort features by importance
imp_list = sorted(zip(model2.feature_importances_, X.columns), reverse=True)
imp_list

In [ ]:
pd.Series(model2.feature_importances_, index=X.columns).nlargest(10).plot(kind='barh')